Evaluation in µDhall is based on applying just three rules:

- Evaluate all function applications by substituting variables.
- Evaluate built-in operations and functions (for example, `10 + 20` or `Natural/subtract 1 2`) according to their specific meaning.
- If the result still contains any lambdas, rename all bound variables to `_` and introduce enough de Bruijn indices to disambiguate name clashes.

Evaluation will take a µDhall expression and produce an equivalent expression in the "normal form". The normal form cannot be evaluated any further. In most cases, the normal form will be a simple value (say, a `Natural` number).

For example, `(λ(y : Natural) → x + 10) 123` is evaluated to the normal form `133`.

However, in some cases the normal form is not a simple value but a function; and it can even be longer than the initial expression.
This is because evaluation in µDhall is _symbolic_. In µDhall, functions are applied even under lambda where some operands are variable symbols.

For example, this expression:

     λ(x : Natural) → (λ(y : Natural) → x + y) 123

is evaluated to the normal form `λ(_ : Natural) → _ + 123`. We have simplified `(λ(y : Natural) → x + y) 123` to `x + 123`, while `x` remains a variable symbol. Then we renamed `x` to `_`, replacing `λ(x : Natural) → x + 123` by `λ(_ : Natural) → _ + 123`.

When bound variables are renamed to `_`, we will introduce de Bruijn indices whenever we have to avoid a name clash. For example:

     λ(a : Type) → λ(b : Type) → a

is renamed to:

     λ(_ : Type) → λ(_ : Type) → _@1

If the variable `_` is already present, we may also need to increment its de Bruijn index:

     λ(x : Type) → _

will become:

     λ(_ : Type) → _@1


Evaluating by substition is called **beta-normalization**. Renaming bound variables is called **alpha-normalization**.

We specify the formal rules for these normalization operations using the "proof notation". We denote by $a\to_\alpha\, b$ a proof that the alpha-normalization of $a$ will give $b$, and by $a\to_\beta\, b$ a proof that the beta-normalization of $a$ will give $b$.

Proof steps are specified for each type of expressions by descending into sub-expressions; this is known as **small-step semantics**.

For example, the beta-normalization rules for adding natural numbers look like this:

$$\frac { a \to_\beta\, \texttt{NaturalLiteral(0)}\quad\quad  b\to_\beta\, b'} { a + b \to_\beta\, b' } $$

$$\frac { a \to_\beta\, a'\quad\quad b\to_\beta\,  \texttt{NaturalLiteral(0)}} { a + b \to_\beta\, a' } $$

$$\frac { a \to_\beta\, \texttt{NaturalLiteral(m)}\quad\quad  b\to_\beta\, \texttt{NaturalLiteral(n)}} { a + b \to_\beta\, \texttt{NaturalLiteral(add m n)} }\quad\textrm{Here ``}\texttt{add}\textrm{'' is the built-in integer addition.} $$

$$ \frac{a \to_\beta\, a'\quad\quad b \to_\beta\, b'}{a + b\to_\beta\, a' + b'} \quad\textrm{If no other rule matches.} $$

These four rules show how to compute the beta-normalization for any expression of the form $a + b$. We will need to specify how these operations work with all possible expression structures. This will allow us to translate the specification directly into code.

Other rules work similarly.

Although the rules are formulated as relations between arbitrary variables, in fact all the rules express functions from input to output.
To clarify this, consider this rule:

$$\frac { a \to_\beta\, a'\quad\quad b\to_\beta \, \texttt{NaturalLiteral(0)}} { a + b \to_\beta\, a' } $$

Formally, this rule says: if any expressions $a$, $a'$, and $b$ happen to be such that $a \to_\beta\, a'$ and $b\to_\beta  \,\texttt{NaturalLiteral(0)}$ then we will have a proof of $a + b \to_\beta\, a'$. So, formally this is just a statement about a relation between arbitrary expressions $a$, $a'$, and $b$. However, we notice that all rules have the form of functions that allow us to compute the right side of $\to\beta$ given variables to the left of $\to_\beta$ by recursive descent into sub-expressions.

Here is an (imaginary) rule that is not of this form:

$$\frac { a \to_\beta\, b\quad\quad c\to_\beta \, b} { a \to_\beta\, c }\quad \text{Not a rule we can use!} $$

The problem with this rule is that we need to _guess_ the expression $c$ such that both $a$ and $c$ are beta-normalized to the same (also initially unknown) expression $b$. Even if we know $b$, the rule does not tell us how to find an expression $c$ such that its beta-normalization returns a given expression $b$. So, this rule cannot be directly translated into an algorithm.

The actual rules of alpha- and beta-normalization were intentionally designed to avoid this situation and to permit a straightforward implementation.